In [ ]:
import pyodbc
import pandas as pd

In [ ]:
pyodbc.drivers()

In [ ]:
conn = pyodbc.connect(
    Driver = '{ODBC Driver 18 for SQL Server}',
    Server = 'localhost',
    Database = 'DbFilme',
    Encrypt = 'Yes',
    TrustServerCertificate = 'Yes',
    UID = 'SA',
    PWD = 'Pg1024_JH', # Sua senha do Banco de Dados Local no SQL Server
    timeout=60
)

cursor = conn.cursor()

In [ ]:
df = pd.read_csv("arquivos/consolidated/movies_consolidado.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
import time

block_size = 500 # quantidade de insert pra não explodir isso ai

# Monte de coisa pra driblar erro
def safe_int(value):
    try:
        if pd.isna(value):
            return None
        return int(value)
    except:
        return None

def safe_float(value):
    try:
        if pd.isna(value):
            return None
        return float(value)
    except:
        return None

def safe_str(value):
    if pd.isna(value):
        return None
    return str(value)

def safe_date(value):
    if pd.isna(value):
        return None
    if value == '0000-00-00':
        return None
    return value

In [ ]:
c = 0

# A parte que insere os gêneros e os filmes
for row in df.itertuples():
    try:
        # Adiciona os gêneros se não existirem
        genres = [g.strip() for g in safe_str(row.genres).split(',')]
        for genre in genres:
            cursor.execute('SELECT COUNT(*) FROM genres WHERE genre_name = ?', genre)
            genre_exists = cursor.fetchone()[0] > 0
            if not genre_exists:
                cursor.execute('INSERT INTO genres (genre_name) VALUES (?)', genre)

        # Verifica se o filme já existe na tabela movies
        cursor.execute('SELECT COUNT(*) FROM movies WHERE movie_id = ?', row.id)
        movie_exists = cursor.fetchone()[0] > 0

        if not movie_exists:
            cursor.execute('''
                INSERT INTO movies (
                    movie_id, title, release_date, description,
                    duration, tagline, metascore, metascore_count,
                    userscore, userscore_count, userscore_sentiment,
                    production_companies, director, writer, top_cast)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                row.id,
                safe_str(row.title),
                safe_date(row.releaseDate),
                safe_str(row.description),
                safe_int(row.duration),
                safe_str(row.tagline),
                safe_int(row.metascore),
                safe_int(row.metascore_count),
                safe_int(row.userscore),
                safe_int(row.userscore_count),
                safe_str(row.userscore_sentiment),
                safe_str(row.production_companies),
                safe_str(row.director),
                safe_str(row.writer),
                safe_str(row.top_cast)
            ))

        # Para cada gênero relacionado ao filme, insere na tabela movie_genres
        cursor.execute('SELECT genres_id FROM genres WHERE genre_name = ?', genre)
        genres_id = cursor.fetchone()[0]

        cursor.execute('''
            INSERT INTO movie_genres (movie_id, genre_id)
            VALUES (?, ?)
        ''', (row.id, genres_id))
        c += 1
        if c % block_size == 0:
            conn.commit()
            time.sleep(0.1)  # pausa para aliviar o servidor

    except Exception as e:
        print(f"Erro ao inserir linha: {row}\n→ {e}")

# Commit final
conn.commit()
print('Finalizado!')


In [ ]:
df = pd.read_csv("arquivos/consolidated/movies_reviews_consolidado.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
for row in df.itertuples():
    try:
        cursor.execute('''
            INSERT INTO review (quote, score, publication_date, author, publication_name, review_type)
            VALUES(?,?,?,?,?,?)
        ''',(
            row.quote,
            row.score,
            row.date,
            row.author,
            row.publicationName,
            row.review_type
        ))
    except Exception as e:
        print("Erro ao inserir linha:", row)
        print(e)

conn.commit()